# Week 7-9 Google Play 크롤링 예제

In [112]:
from selenium import webdriver
import pandas as pd
import time
 
url = "https://play.google.com/store/apps/details?id=com.lgeha.nuts&showAllReviews=true" 
driverPath = "D:\kjw\lecture\LG전자_21\week7_자연어처리\chromedriver.exe" 
driver = webdriver.Chrome(driverPath) 
driver.get(url)

In [13]:
# 리뷰
comment = driver.find_element_by_xpath("//span[@jsname='bN97Pc']") 
comment.text

'미니워시 상황 클릭하면 업데이트 하라고 뜨는데, 업데이트 클릭하면 제품이 작동 중일 때는 작동 중이라 안 된다고 뜨고 작동 중이 아닐 땐 전원이 꺼져 있으니 켜져 있을 때 다시 하라 합니다. 무한반복.'

In [14]:
# 날짜
date = driver.find_element_by_xpath("//span[@class='p2TkOb']") 
date.text

'2021년 9월 25일'

In [15]:
# 좋아요 수
like = driver.find_element_by_xpath("//div[@aria-label='이 리뷰가 유용하다는 평가를 받은 횟수입니다.']") 
like.text

'1'

In [16]:
# 별점 데이터
star = driver.find_element_by_xpath("//span[@class='nt2C1d']/div[@class='pf5lIe']/div[@role='img']") 
star.get_attribute('aria-label')

'별표 5개 만점에 4개를 받았습니다.'

In [17]:
star.get_attribute('aria-label')[10]

'4'

In [118]:
comments = driver.find_elements_by_xpath("//span[@jsname='bN97Pc']") 
len(comments)

680

In [113]:
driver.find_element_by_xpath("//span[@class='RveJvd snByac']").click()

In [114]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

# 5번 스크롤 내리기
for i in range(5): 
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)") 
    time.sleep(3)

# 더보기 누르기
element=driver.find_element_by_xpath("//span[@class='RveJvd snByac']")
driver.execute_script("arguments[0].click();", element)


In [103]:
SCROLL_PAUSE_TIME = 3 
last_height = driver.execute_script("return document.body.scrollHeight") 
while True: 
    # (1) 5번의 스크롤링 
    for i in range(5): 
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") 
        time.sleep(SCROLL_PAUSE_TIME) 
    # (2) 더보기 클릭 
    try:
        driver.find_element_by_xpath("//span[@class='RveJvd snByac']").click()
    except:
        break
    # (3) 종료 조건 
    new_height = driver.execute_script("return document.body.scrollHeight") 
    if new_height == last_height: 
        break 
    last_height = new_height

In [115]:
# 편의상 한 번 만 반복하기
for i in range(5): 
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") 
    time.sleep(SCROLL_PAUSE_TIME) 

    # (2) 더보기 클릭 
driver.find_element_by_xpath("//span[@class='RveJvd snByac']").click()

In [107]:
# 주어진 횟수만큼만 더보기 하도록 수정한 버전

SCROLL_PAUSE_TIME = 4 
MAX_ITER = 100
last_height = driver.execute_script("return document.body.scrollHeight") 
iter = 0
while True: 
    # (1) 5번의 스크롤링 
    for i in range(5): 
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") 
        time.sleep(SCROLL_PAUSE_TIME)
    # (2) 더보기 클릭 
    try:
        driver.find_element_by_xpath("//span[@class='RveJvd snByac']").click()
    except:
        print("Fail to click")
        break
    # (3) 종료 조건 
    new_height = driver.execute_script("return document.body.scrollHeight") 
    if new_height == last_height: 
        break 
    last_height = new_height

    if iter > 100:
        break
    else:
        print("Iteration:"+str(iter),end=" ")
        iter=iter+1

Iteration:0 Iteration:1 Iteration:2 Iteration:3 Iteration:4 Iteration:5 Iteration:6 Iteration:7 Iteration:8 Iteration:9 Iteration:10 Iteration:11 Iteration:12 Iteration:13 Iteration:14 Iteration:15 Iteration:16 Iteration:17 Iteration:18 Iteration:19 Iteration:20 Fail to click


In [119]:
# 데이터 가져오기

comments = driver.find_elements_by_xpath("//span[@jsname='bN97Pc']") 
len(comments)
# 날짜
dates = driver.find_elements_by_xpath("//span[@class='p2TkOb']") 
len(dates)
# 좋아요 수

likes = driver.find_elements_by_xpath("//div[@aria-label='이 리뷰가 유용하다는 평가를 받은 횟수입니다.']") 
len(likes)
# 별점 데이터

stars = driver.find_elements_by_xpath("//span[@class='nt2C1d']/div[@class='pf5lIe']/div[@role='img']") 
len(stars)


680

In [120]:
# 데이터프레임에 저장하기
# Make a dataframe
res_dict = [] 
for i in range(len(comments)): 
    res_dict.append({ 
        'date' : dates[i].text, 
        'star' : stars[i].get_attribute('aria-label')[10], 
        'like' : likes[i].text, 
        'comment' : comments[i].text })
res_df = pd.DataFrame(res_dict) 
res_df

,date,star,like,comment
0,2021년 9월 25일,4,1,"미니워시 상황 클릭하면 업데이트 하라고 뜨는데, 업데이트 클릭하면 제품이 작동 중일..."
1,2021년 9월 27일,5,8,"아카라허브 연결 후 문열림센서를 연결하려고 하는데요, 허브에서는 등록이 잘 되었다고..."
2,2021년 9월 26일,3,6,뜻하지않게 여러개의 엘지 제품이 여러개등록해서쓰고있는데요 에어컨이나 공기청정기는 어...
3,2021년 9월 24일,1,1,TV로 모바일 컨텐츠보기가 업뎃되면서 바뀌었는데 진심 너무너무 불편해요. 전엔 연결...
4,2021년 9월 27일,1,1,그놈의 제품 추가정보등록... 폰바꿨다고 다시 등록해야하고 뭐때문에 또 다시 등록해...
...,...,...,...,...
675,2021년 6월 10일,1,9,어느순간 제품이 연결해제되서 안켜짐. 결국 다 지우고 다시 등록하는데 제품와이파이 ...
676,2021년 6월 17일,1,10,업데이트 이후 등록된 제품 원격제어가 갑자기 불가능해져서 전부 수동으로 새로 연결....
677,2021년 5월 16일,2,2,"집에 세탁기, 건조기, 에어컨 2대, 제습기, 스타일러, 정수기까지 총 7대 모두 ..."
678,2021년 6월 21일,1,8,엘지가 왜 만년 2등인지 알 수 있게 해주는 앱. 담당 부서 직원들도 집에서 이 앱...


In [34]:
# csv 파일에 저장하기
import os
save_file_path = 'D:\kjw\lecture\LG전자_21\week7_자연어처리'
os.chdir(save_file_path)
res_df.to_csv('thinq_review.csv', index=False, encoding='utf-8-sig')